In [1]:
import xml.etree.ElementTree as ET

In [2]:
def convert_to_yolo(size, box):
    """
    Преобразует координаты ограничивающего прямоугольника из формата XML в формат YOLO.
    
    size: размер изображения (ширина, высота)
    box: координаты ограничивающего прямоугольника (xmin, xmax, ymin, ymax)
    
    Возвращает: координаты в формате YOLO (x_center, y_center, width, height)
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (box[0] + box[1]) / 2.0
    y_center = (box[2] + box[3]) / 2.0
    width = box[1] - box[0]
    height = box[3] - box[2]
    return (x_center * dw, y_center * dh, width * dw, height * dh)

In [3]:
def convert_xml_to_yolo(xml_file, output_dir):
    """
    Конвертирует аннотацию XML в формат YOLO.
    
    xml_file: путь к XML файлу
    output_dir: директория для сохранения YOLO файлов
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name == 'crazing':
            class_id = 0  # Здесь можно указать id класса для "crazing"
        elif name == 'inclusion':
            class_id = 1
        elif name == 'patches':
            class_id = 2
        elif name == 'pitted_surface':
            class_id = 3
        elif name == 'rolled-in_scale':
            class_id = 4
        elif name == 'scratches':
            class_id = 5
        else:
            continue
        
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        box = (xmin, xmax, ymin, ymax)
        yolo_box = convert_to_yolo((width, height), box)
        
        filename = root.find('filename').text
        yolo_filename = f"{output_dir}/{filename.split('.')[0]}.txt"
        
        with open(yolo_filename, 'w') as f:
            f.write(f"{class_id} {' '.join(map(str, yolo_box))}\n")


In [ ]:
/home/andrew/works/shpad_add/Detection-of-welding-seams/archive/train_annotations/crazing_2.xml

In [4]:
convert_xml_to_yolo('/home/andrew/works/shpad_add/Detection-of-welding-seams/archive/train_annotations/crazing_2.xml', '/home/andrew/works/shpad_add/Detection-of-welding-seams/notebooks')

In [7]:
import os

In [5]:
def convert_xml_to_yolo(xml_file, output_dir):
    """
    Конвертирует аннотацию XML в формат YOLO.
    
    xml_file: путь к XML файлу
    output_dir: директория для сохранения YOLO файлов
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    annotations = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name == 'crazing':
            class_id = 0
        elif name == 'inclusion':
            class_id = 1
        elif name == 'patches':
            class_id = 2
        elif name == 'pitted_surface':
            class_id = 3
        elif name == 'rolled-in_scale':
            class_id = 4
        elif name == 'scratches':
            class_id = 5
        else:
            continue
        
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        box = (xmin, xmax, ymin, ymax)
        yolo_box = convert_to_yolo((width, height), box)
        annotations.append(f"{class_id} {' '.join(map(str, yolo_box))}")
    
    if annotations:
        filename = root.find('filename').text
        yolo_filename = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.txt")
        
        with open(yolo_filename, 'w') as f:
            f.write("\n".join(annotations))

In [6]:
def convert_directory(input_dir, output_dir):
    """
    Конвертирует все XML файлы из входной директории в YOLO формат и сохраняет в выходной директории.
    
    input_dir: директория с XML файлами
    output_dir: директория для сохранения YOLO файлов
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        if file_name.endswith('.xml'):
            xml_file = os.path.join(input_dir, file_name)
            convert_xml_to_yolo(xml_file, output_dir)

In [8]:
input_directory = '/home/andrew/works/shpad_add/Detection-of-welding-seams/archive/train_annotations'  # Замените на путь к вашей директории с XML файлами
output_directory = '/home/andrew/works/shpad_add/Detection-of-welding-seams/archive/yolo_tr_annotations'  # Замените на путь к директории для сохранения YOLO файлов

convert_directory(input_directory, output_directory)